# XGBoost Parameter Tuning for Rent Listing Inqueries

Rental Listing Inquiries数据集是Kaggle平台上的一个分类竞赛任务，需要根据公寓的特征来预测其受欢迎程度（用户感兴趣程度分为高、中、低三类）。其中房屋的特征x共有14维，响应值y为用户对该公寓的感兴趣程度。评价标准为logloss。 数据链接：https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries
第5步：调整正则化参数：reg_lambda

行列采样参数略


In [1]:
from xgboost import XGBClassifier

import pandas as pd 
import numpy as np

import math

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import log_loss

from matplotlib import pyplot
%matplotlib inline

# 读取数据

In [2]:
train = pd.read_csv("RentListingInquries_FE_train.csv")
#train.head()

y_train = train['interest_level']

train = train.drop([ "interest_level"], axis=1)
X_train = train

# 此前已经调好的参数： n_estimators：238 max_depth：6 min_child_weight：5 reg_alpha：1

用交叉验证评价模型性能时，用scoring参数定义评价指标。评价指标是越高越好，因此用一些损失函数当评价指标时，需要再加负号，如neg_log_loss，neg_mean_squared_error 详见sklearn文档：http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss


In [3]:
#reg_alpha = [ 0.1, 1, 2]    #default = 0, 测试0.1,1，1.5，2
reg_lambda = [2, 2.5, 3]      #default = 1，测试0.1， 0.5， 1，2

param_test5 = dict(reg_lambda=reg_lambda)
param_test5

{'reg_lambda': [2, 2.5, 3]}

In [4]:
xgb5 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=238,  #第二轮参数调整得到的n_estimators最优值
        max_depth=6,
        min_child_weight=5,
        gamma=0,
        subsample=0.5,
        colsample_bytree=0.8,
        colsample_bylevel = 0.7,
        reg_alpha = 1,
        objective= 'multi:softprob',
        seed=3)


gsearch5 = GridSearchCV(xgb5, param_grid = param_test5, scoring='neg_log_loss',n_jobs=-1, cv=3)
gsearch5.fit(X_train , y_train)

gsearch5.grid_scores_, gsearch5.best_params_,     gsearch5.best_score_

/home/shnu/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.57736, std: 0.00406, params: {'reg_lambda': 2},
  mean: -0.57784, std: 0.00461, params: {'reg_lambda': 2.5},
  mean: -0.57830, std: 0.00413, params: {'reg_lambda': 3}],
 {'reg_lambda': 2},
 -0.5773587602956511)